In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1488bea2-0d4e-5df3-2a09-0919ecd83729)


## IMPORTING LIBRARIES

In [26]:
import tensorflow as tf
import keras 
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import sys
import random
import pandas as pd

from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

## LOADING IMAGES


In [143]:
train_img_dir= "/content/drive/MyDrive/schneider/train_test_data/train"
test_img_dir= "/content/drive/MyDrive/schneider/train_test_data/test"

train_csv="/content/drive/MyDrive/schneider/train.csv"
test_csv="/content/drive/MyDrive/schneider/test.csv"


traindf=pd.read_csv(train_csv)
testdf=pd.read_csv(test_csv)
traindf["label"]=traindf["label"].apply(str)


 DATA CLEANING

In [144]:
def nombrestrain(string):
  return string[22:]
def nombrestest(string):
  return string[21:]

traindf["id"]= traindf["example_path"].apply(nombrestrain)
testdf["id"]= testdf["example_path"].apply(nombrestest)
traindf = traindf[["id", "label"]]
testdf=testdf[["id"]]

CHECK if CLASSES are BALANCED  --> They are not

In [145]:
traindf0 = traindf[traindf["label"]=="0"].reset_index()[["id", "label"]]
traindf1 = traindf[traindf["label"]=="1"].reset_index()[["id", "label"]]
traindf2 = traindf[traindf["label"]=="2"].reset_index()[["id", "label"]]

print([len(traindf0), len(traindf1), len(traindf2)]) #CLASSES ARE UNBALANCED

[860, 196, 658]


### BALANCING CLASES

In [146]:


balanceado0= traindf0
balanceado1= traindf1.append(traindf1).append(traindf1).append(traindf1).append(traindf1[:76])
balanceado2= traindf2.append(traindf2[:202])

traindf= balanceado0.append(balanceado1).append(balanceado2).append(balanceado2)

In [147]:
[len(balanceado0),len(balanceado1),len(balanceado2)]

[860, 860, 860]

In [148]:
[len(traindf)]

[3440]

CROPPING FUNCTION for later...

In [149]:
def rand_crop(img):
    # Select size of crop
    h = random.randint(input_size, image_size) 
    # Select lower left corner for crop
    cx = random.randint(0, image_size-h)
    cy = random.randint(0, image_size-h)
    # Crop and resize image to input_size
    cropped_img = img[cx:cx+h,cy:cy+h,:]
    return cv2.resize(cropped_img, (input_size,input_size))


# IMAGE DATA GENERATOR

In [150]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.01) # I want all the data for the final prediction, just 34 images for validation because lack of time for fitting 2 models
image_size = 1024
input_size = 1024


train_gen=datagen.flow_from_dataframe(
dataframe=traindf, 
directory=train_img_dir, 
preprocessing_function = rand_crop,
x_col="id",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))


valid_gen=datagen.flow_from_dataframe(
dataframe=traindf,
directory=train_img_dir,
x_col="id",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))


test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory=test_img_dir,
x_col="id",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))


classes = dict((v, k) for k, v in train_gen.class_indices.items())
num_classes = len(classes)

Found 3406 validated image filenames belonging to 3 classes.
Found 34 validated image filenames belonging to 3 classes.
Found 286 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 349 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [151]:
num_classes

3

DEFINING METRICS for the model




In [152]:
from keras import backend as K
    
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
  
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [153]:
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.layers import GlobalMaxPooling2D, Dense
from keras.optimizers import Adam

# Create a VGG19 architecture
pretrained_model = VGG19(include_top=False,
                         
                         input_shape=(input_size, input_size, 3),
                         weights='imagenet')
x = GlobalMaxPooling2D()(pretrained_model.output)

x = Dense(2048, activation='relu')(x)
x = Dense(2048, activation='relu')(x)

output = Dense(num_classes, activation='softmax')(x)
vgg19_model = Model(pretrained_model.input, output)


In [154]:

# Create new model with modified config which accepts the input shape: [input_size, input_size, 1]
cfg = vgg19_model.get_config()
cfg['layers'][0]['config']['batch_input_shape'] = (None, input_size, input_size, 3)

model = Model.from_config(cfg)

for i, layer in enumerate(model.layers):
    if i == 1:
        layer.trainable = False
    else:
        layer.trainable = True 
        layer.set_weights(vgg19_model.layers[i].get_weights())

# Compile the model with Adam optimizer, binary crossentropy loss and f1 score as the metric
adam = Adam(lr=0.000005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) # 10x smaller than standard
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=[f1])
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1024, 1024, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, 1024, 1024, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 1024, 1024, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 512, 512, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 512, 512, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 512, 512, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 256, 256, 128)     0   

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


### TEST MODEL FOR SEEING F1 score with validation  Manually interrupted 

In [155]:
history = model.fit((train_gen),
                              epochs=300,batch_size=20 ,
                              steps_per_epoch=13, # Effectively 1 run through every possibility of reflected data
                              validation_data=valid_gen,
                              validation_steps=len(valid_gen))

Epoch 1/300
13/13 [==============================] - 5s 271ms/step - loss: 0.6509 - f1: 0.0000e+00 - val_loss: 0.7423 - val_f1: 0.0000e+00
Epoch 2/300
13/13 [==============================] - 3s 224ms/step - loss: 0.5987 - f1: 0.4255 - val_loss: 0.8432 - val_f1: 0.0000e+00
Epoch 3/300
13/13 [==============================] - 3s 223ms/step - loss: 0.6051 - f1: 0.5096 - val_loss: 0.8431 - val_f1: 0.0000e+00
Epoch 4/300
13/13 [==============================] - 3s 223ms/step - loss: 0.6202 - f1: 0.4663 - val_loss: 0.7746 - val_f1: 0.0000e+00
Epoch 5/300
13/13 [==============================] - 3s 225ms/step - loss: 0.6063 - f1: 0.5072 - val_loss: 0.7948 - val_f1: 0.0000e+00
Epoch 6/300
13/13 [==============================] - 3s 223ms/step - loss: 0.5995 - f1: 0.5168 - val_loss: 0.8252 - val_f1: 0.0000e+00
Epoch 7/300
13/13 [==============================] - 3s 221ms/step - loss: 0.6131 - f1: 0.4808 - val_loss: 0.8516 - val_f1: 0.0000e+00
Epoch 8/300
13/13 [==============================] 

PREDICTING EVALUATION DATASET

In [156]:
predIdxs = model.predict_generator(valid_gen)

predIdxs = np.argmax(predIdxs, axis=1)

predIdxs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


array([0, 0, 2, 1, 0, 0, 2, 1, 0, 2, 2, 0, 2, 1, 0, 0, 1, 1, 1, 2, 2, 1,
       2, 0, 1, 1, 2, 0, 0, 1, 0, 2, 0, 0])

# PREDICTING TEST DATASET

In [157]:
filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict(test_generator,nb_samples)



predIdxs = np.argmax(predict, axis=1)

9/9 [==============================] - 2s 201ms/step


In [173]:
results=pd.DataFrame({"id": filenames, "target": predIdxs})

In [174]:
results

,id,target
0,1001.png,1
1,1445.png,1
2,1530.png,2
3,1.png,2
4,1800.png,2
...,...,...
281,1878.png,2
282,1294.png,2
283,1276.png,2
284,1117.png,1


In [175]:
#WE WILL JUST SEND THE TARGET

results["target"]

0      1
1      1
2      2
3      2
4      2
      ..
281    2
282    2
283    2
284    1
285    2
Name: target, Length: 286, dtype: int64

In [176]:
results.to_csv("predictions.csv") #EXPORTAMOS LOS RESULTADOS csv

In [177]:
results["target"].to_json("predictions.json") #EXPORTAMOS LOS RESULTADOS JSON